In [8]:
import cv2 as cv

In [9]:
metodo = 'MOG2'
#metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = True)

In [10]:
# Abrimos el archivo
#-------------------
filename = 'bridge.avi'
capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
    fgMask = backSub.apply(frame)
    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # mostramos frame original e imagen binaria background/foreground
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

In [15]:
##El metodo MOG2, el video es más rápido que usando el método Knn
##esto se debe a que el tiempo de clasificación de Knn es mucho mas lento
##ya que debe realizar mediciones entre vecinos, mientras que MOG2 solo genera
#una probabilidad inmediata basado en lo que entrenó inicialmente.

##MOG2 genera el stream en la velocidad correcta pero con más ruido
##Knn genera el stream en velocidad mas lenta pero sin tanto ruido

##Adicional existe un "ruido" generado por el agua, tiene un movimiento
##pero su color cambia demasiado entre frame por la luz que refleja del
##sol, este detalle es omitido por Knn pero si es tomado por MOG2, en
##algunos casos este "ruido" podría ser necesario si deseamos ser mas
##detallistas